In [1]:
import sys
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt



path='/home/jbohne/sbu/'
sys.path.append(f'{path}taq_db/TAQ-Query-Scripts/data')
sys.path.append(f'{path}research/Adaptive_Signal_Estimation_Private')

from trend_filtering.adaptive_tf import adaptive_tf
from trend_filtering.test_adaptive_tf import test_adaptive_tf
from profiling.profiler import profile

In [2]:
trades=pd.read_csv(f'{path}taq_db/TAQ-Query-Scripts/data/AAPL_trades.csv',index_col=0).dropna()

price=trades['Trade_Price'].values

### Overview of Estimation

In [3]:
test_adaptive_tf(price,1000,include_cv=True,lambda_p=1.0,plot=True,verbose=True)



solved
pobj1: 6713226.6070587775, pobj2: 1.025588783582571, dobj: 1.0255879704727884, gap: 8.131097826158395e-07
solved
pobj1: 540.2268825234457, pobj2: 25.801914881168234, dobj: 25.80191392945619, gap: 9.517120425073244e-07
solved
pobj1: 371.8392637575362, pobj2: 36.291055940937085, dobj: 36.29105528240609, gap: 6.585309932916061e-07


### Performance of Sherman-Morrison -Woodbury Formula vs Naive Matrix Inversion

Naive Matrix Inversion

In [ ]:
profile(test_adaptive_tf)(price,1000,include_cv=False,lambda_p=1.0,plot=False,verbose=False)

Profiling: test_adaptive_tf


: 

: 

Sherman-Morrison -Woodbury Formula

In [ ]:
profile(test_adaptive_tf)(price,1000,include_cv=False,lambda_p=1.0,plot=False,verbose=False)

### Numba Compilation

In [3]:
profile(test_adaptive_tf)(price,1000,include_cv=False,lambda_p=1.0,plot=False,verbose=False)

Profiling: test_adaptive_tf

*** PROFILER RESULTS ***
inner (/home/jbohne/sbu/research/Adaptive_Signal_Estimation_Private/profiling/profiler.py:10)
function called 1 times

         2685580 function calls (2524902 primitive calls) in 3.972 seconds

   Ordered by: cumulative time, internal time, call count
   List reduced from 3463 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.976    3.976 profiler.py:10(inner)
        1    0.000    0.000    3.976    3.976 test_adaptive_tf.py:10(test_adaptive_tf)
        1    0.000    0.000    3.936    3.936 adaptive_tf.py:9(adaptive_tf)
        2    0.000    0.000    3.935    1.967 dispatcher.py:388(_compile_for_args)
     47/2    0.001    0.000    3.900    1.950 dispatcher.py:915(compile)
     11/2    0.000    0.000    3.433    1.716 dispatcher.py:124(compile)
     11/2    0.000    0.000    3.433    1.716 dispatcher.py:131(_compile_cached)
     11/2    0.000    0

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function abs>) found for signature:
 
 >>> abs(array(float64, 2d, C))
 
There are 6 candidate implementations:
  - Of which 2 did not match due to:
  Type Restricted Function in function 'abs': File: unknown: Line unknown.
    With argument(s): '(array(float64, 2d, C))':
   No match for registered cases:
    * (int8,) -> int8
    * (int16,) -> int16
    * (int32,) -> int32
    * (int64,) -> int64
    * (uint8,) -> uint8
    * (uint16,) -> uint16
    * (uint32,) -> uint32
    * (uint64,) -> uint64
    * (float32,) -> float32
    * (float64,) -> float64
    * (complex64,) -> float32
    * (complex128,) -> float64
  - Of which 4 did not match due to:
  Overload of function 'abs': File: <numerous>: Line N/A.
    With argument(s): '(array(float64, 2d, C))':
   No match.

During: resolving callee type: Function(<built-in function abs>)
During: typing of call at /home/jbohn